In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

video_path1 = '../data/leaves.mp4'
video_path2 = '../data/cars.mp4'

#### Visualization and Analysis

In [7]:
blob_data = []
frame_count = 0

cap = cv2.VideoCapture(video_path2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                   cv2.THRESH_BINARY, 11, 2)
    thresh = cv2.bitwise_not(thresh)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    MIN_AREA = 500
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > MIN_AREA:
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                blob_data.append({
                    'frame': frame_count,
                    'center_x': cX,
                    'center_y': cY,
                    'area': area
                })

    frame_count += 1

cap.release()

# Create DataFrame
import pandas as pd
df = pd.DataFrame(blob_data)
df.to_csv('../data/CSV Files/blob_data.csv', index = False)
# Show DataFrame
print(df.head())


   frame  center_x  center_y    area
0      0      1441      1062   631.5
1      0        53       961   633.5
2      0        93       968  1088.0
3      0       222       949   623.0
4      0      1092       935   517.0
